<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-14 14:14:42
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: -14.72 K (-0.11%)
Current PnL: -21.99 L (-14.96%)
CY Booked + Current PnL: -13.72 L (-9.33%)
-------------------
Total profit:  2.59 L
Total loss:  -24.58 L
-------------------
Total Booked + Current PnL: 13.26 L (11.04%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.22%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.47 L (61.31%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBIN,760.30,863.00,-11.34,M-LC,3.83,219221.0,17742.0,9470.0,0.66,8.81,4.32,13.51,61.0,1.87,1.68,22.51,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-19.90,H-LC,6.20,110680.0,6368.0,10282.0,-1.25,6.10,9.29,15.96,11.0,0.62,0.85,24.03,X5K,ATH,METALS
4,APOLLOHOSP,7098.95,8285.00,-13.97,H-LC,5.93,171952.0,15775.0,10317.0,0.10,10.10,6.00,16.71,19.0,1.53,1.32,29.13,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,11.18,H-MC,13.81,158804.0,32604.0,13578.0,0.99,25.84,8.55,36.59,111.0,2.40,1.22,41.59,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-25.00,H-SC,9.64,224087.0,31067.0,16963.0,1.21,16.10,7.57,24.89,134.0,1.83,1.72,43.20,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,RAJOOENG,102.50,143.33,-32.60,H-SC,4.61,102100.0,-400.0,41228.0,1.09,-0.39,40.38,39.83,116.0,-0.01,0.78,1.09,AR,ATH,MISC
25,DABUR,505.20,735.00,-10.14,H-MC,3.47,195390.0,-1638.0,91267.0,-0.50,-0.83,46.71,45.49,103.0,-0.02,1.50,12.10,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-9.05,H-MC,7.72,190845.0,-897.0,96930.0,-0.30,-0.47,50.79,50.08,101.0,-0.01,1.46,6.30,XY25,ATH,AC
58,PGHH,13388.00,18062.58,-31.08,H-MC,6.35,199530.0,-1290.0,71412.0,1.54,-0.64,35.79,34.92,82.0,-0.02,1.53,4.00,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-19.78,H-LC,2.53,214213.0,9713.0,31704.0,0.17,4.75,14.80,20.25,10.0,0.31,1.64,13.90,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-12.15,H-LC,3.19,171374.0,-30012.0,117631.0,0.80,-14.90,68.64,43.51,27.0,-0.26,1.31,19.29,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-17.13,H-LC,3.24,127413.0,-138013.0,194916.0,-0.72,-52.00,152.98,21.44,8.0,-0.71,0.98,2.38,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-6.65,H-LC,3.73,185472.0,-14838.0,74819.0,0.73,-7.41,40.34,29.94,17.0,-0.20,1.42,28.13,X200,ATH,IT
37,HINDUNILVR,2413.81,2723.0,-11.65,H-LC,5.77,277894.0,7547.0,27095.0,-0.56,2.79,9.75,12.81,28.0,0.28,2.13,16.22,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,VALIANTORG,512.64,838.0,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.74,60.62,XR,NTT,CHEMICALS
10,BAJAJHFL,122.26,152.0,-12.15,H-MC,5.31,185320.0,-15186.0,63954.0,4.09,-7.57,34.51,24.33,98.0,-0.24,1.42,6.87,X40N,BTT,FINANCE
29,EASEMYTRIP,18.12,26.4,-7.85,M-SC,24.82,88700.0,-88115.0,168911.0,2.60,-49.83,190.43,45.70,197.0,-0.52,0.68,3.18,XY24,NTT,TRAVEL
30,GREENPANEL,375.16,537.0,201.03,M-SC,5.70,134346.0,-48732.0,127709.0,2.51,-26.62,95.06,43.14,230.0,-0.38,1.03,26.37,XY24,NTT,MISC
92,WIPRO,243.46,420.0,-12.89,M-LC,7.31,153407.0,2462.0,106986.0,2.39,1.63,69.74,72.51,54.0,0.02,1.18,7.75,XR,NTT,IT


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.00,-42.44,M-SC,7.24,86030.0,-42640.0,87940.0,-5.51,-33.14,102.22,35.20,209.0,-0.48,0.66,0.00,X40,NTT,FOOTWEAR
65,SAIL,130.64,228.00,31.52,M-MC,10.18,207553.0,-17409.0,185054.0,-2.84,-7.74,89.16,74.53,193.0,-0.09,1.59,20.51,XY24,BTT,STEEL
70,SHALBY,261.39,327.00,933.49,M-SC,6.89,140617.0,-40526.0,86001.0,-2.62,-22.37,61.16,25.10,232.0,-0.47,1.08,12.69,XY24,NTT,HEALTHCARE
89,VBL,492.64,671.64,-11.16,H-LC,10.14,269646.0,4606.0,91707.0,-1.94,1.74,34.01,36.33,5.0,0.05,2.07,15.27,X40N,ATH,FMCG
84,UJJIVANSFB,52.77,60.00,78.53,H-SC,20.41,113103.0,-29376.0,48894.0,-1.94,-20.62,43.23,13.70,163.0,-0.60,0.87,33.49,OX40N,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,92.74,M-SC,10.34,83067.0,-17710.0,17801.0,-0.15,-17.57,21.43,0.09,243.0,-0.99,0.64,9.13,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,78.53,H-SC,20.41,113103.0,-29376.0,48894.0,-1.94,-20.62,43.23,13.70,163.0,-0.60,0.87,33.49,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-20.05,H-SC,3.10,114111.0,-29692.0,56439.0,-1.11,-20.65,49.46,18.60,148.0,-0.53,0.88,25.92,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,109.82,M-SC,2.91,137528.0,-37031.0,37064.0,0.90,-21.21,26.95,0.02,221.0,-1.00,1.05,19.26,OX40N,NTT,PAINTS
49,KANSAINER,299.63,340.0,-69.25,H-SC,2.82,212193.0,-57474.0,93811.0,-0.30,-21.31,44.21,13.47,142.0,-0.61,1.63,7.32,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.69,H-LC,15.97,235950.0,-59984.0,153887.0,-0.37,-20.27,65.22,31.73,2.0,-0.39,1.81,0.73,X40,BTT,IT
45,INFY,1461.46,2275.00,-20.67,H-LC,10.91,259818.0,-1783.0,147395.0,1.75,-0.68,56.73,55.67,4.0,-0.01,1.99,5.38,X40,BTT,IT
89,VBL,492.64,671.64,-11.16,H-LC,10.14,269646.0,4606.0,91707.0,-1.94,1.74,34.01,36.33,5.0,0.05,2.07,15.27,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.10,H-LC,10.47,196599.0,-19920.0,112828.0,-0.90,-9.20,57.39,42.91,7.0,-0.18,1.51,2.96,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-17.13,H-LC,3.24,127413.0,-138013.0,194916.0,-0.72,-52.00,152.98,21.44,8.0,-0.71,0.98,2.38,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,11.18,H-MC,13.81,158804.0,32604.0,13578.0,0.99,25.84,8.55,36.59,111.0,2.4,1.22,41.59,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,11.18,H-MC,13.81,158804.0,32604.0,13578.0,0.99,25.84,8.55,36.59,111.0,2.40,1.22,41.59,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,7.49,H-SC,12.21,128268.0,2971.0,56027.0,-1.25,2.37,43.68,47.09,141.0,0.05,0.98,60.54,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,3.51,H-MC,6.93,178620.0,13950.0,54015.0,1.73,8.47,30.24,41.28,91.0,0.26,1.37,17.13,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-4.72,H-SC,3.87,108405.0,6401.0,37530.0,1.31,6.28,34.62,43.06,123.0,0.17,0.83,24.47,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.82,165291.0,14378.0,42926.0,-0.97,9.53,25.97,37.97,92.0,0.33,1.27,37.24,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.82,165291.0,14378.0,42926.0,-0.97,9.53,25.97,37.97,92.0,0.33,1.27,37.24,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,11.98,135055.0,11719.0,122049.0,-0.18,9.50,90.37,108.45,119.0,0.10,1.04,33.98,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,7.49,H-SC,12.21,128268.0,2971.0,56027.0,-1.25,2.37,43.68,47.09,141.0,0.05,0.98,60.54,MH,ATH,POWER
92,WIPRO,243.46,420.00,-12.89,M-LC,7.31,153407.0,2462.0,106986.0,2.39,1.63,69.74,72.51,54.0,0.02,1.18,7.75,XR,NTT,IT
91,WHIRLPOOL,1219.98,2270.00,-44.96,M-SC,4.28,92962.0,1464.0,77289.0,-0.15,1.60,83.14,86.07,219.0,0.02,0.71,34.48,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.62,73267.0,-40282.0,80330.0,0.16,-35.48,109.64,35.27,268.0,-0.50,0.56,79.32,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.17,H-SC,15.58,89229.0,-11734.0,106263.0,-0.86,-11.62,119.09,93.63,149.0,-0.11,0.68,32.91,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,13.84,91080.0,-6900.0,28772.0,-1.35,-7.04,31.59,22.32,152.0,-0.24,0.70,32.10,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-325.20,H-SC,16.79,95823.0,-37463.0,122059.0,5.00,-28.11,127.38,63.47,143.0,-0.31,0.74,60.62,XR,NTT,CHEMICALS
36,HINDALCO,651.95,756.01,-19.90,H-LC,6.20,110680.0,6368.0,10282.0,-1.25,6.10,9.29,15.96,11.0,0.62,0.85,24.03,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-25.00,H-SC,9.64,224087.0,31067.0,16963.0,1.21,16.10,7.57,24.89,134.0,1.83,1.72,43.20,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-11.08,H-MC,7.82,165291.0,14378.0,42926.0,-0.97,9.53,25.97,37.97,92.0,0.33,1.27,37.24,AR,ATH,AUTO
23,COFFEEDAY,59.14,80.00,-49.70,L-SC,21.62,73267.0,-40282.0,80330.0,0.16,-35.48,109.64,35.27,268.0,-0.50,0.56,79.32,XR,NTT,HOTELS
70,SHALBY,261.39,327.00,933.49,M-SC,6.89,140617.0,-40526.0,86001.0,-2.62,-22.37,61.16,25.10,232.0,-0.47,1.08,12.69,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.00,-14.13,H-LC,6.55,214390.0,29909.0,40455.0,0.94,16.21,18.87,38.14,35.0,0.74,1.64,28.54,X40N,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.43
1,25,41.15
2,50,70.99


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.33
LC    34.37
MC    26.32
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.83
X40N     15.62
X40      13.37
XY25     10.65
XR        9.08
AR        8.43
OX40N     7.49
X5K       2.27
MH        1.74
X200      1.42
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.98
H-SC    24.51
H-MC    23.39
M-SC    13.33
M-LC     7.30
M-MC     2.59
L-SC     1.49
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.19,-11.41,47.23
IT,12.02,-18.96,78.37
FINANCE,8.28,-20.92,65.65
BANKS,7.63,-14.85,63.62
MISC,5.92,-19.17,87.48
PAINTS,5.65,-18.44,41.79
HEALTHCARE,5.34,5.60,18.24
ELECTRICAL,4.74,-13.63,62.84
AUTO,4.49,-15.50,65.76


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2880391.0,23
XR,1111849.0,13
AR,1110863.0,10
X40,827331.0,10
XY25,660589.0,8
X40N,570034.0,12
OX40N,483386.0,10
SR,196563.0,2
X5K,142826.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2823166.0,28
M-SC,1509070.0,17
H-LC,1348544.0,18
H-MC,1328489.0,18
M-LC,427322.0,5
M-MC,339743.0,2
L-SC,267361.0,3
L-MC,58634.0,1
L-LC,44517.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      947547.0      6
M-SC       XY24      875729.0      7
H-SC       AR        674519.0      5
           XR        524503.0      6
H-LC       X40       513409.0      5
H-MC       XY24      498096.0      5
M-MC       XY24      339743.0      2
H-LC       XY25      259677.0      3
           AR        232657.0      2
H-MC       X40       225982.0      4
           XY25      222536.0      2
H-LC       X40N      216746.0      5
H-SC       X40N      213237.0      4
           OX40N     210770.0      4
           SR        196563.0      2
M-LC       XY24      178322.0      2
L-SC       XR        176806.0      2
M-SC       AR        160761.0      2
H-MC       X40N      140051.0      3
M-SC       OX40N     133365.0      4
M-LC       X5K       132544.0      1
M-SC       XR        126886.0      2
           XY25      124389.0      1
H-MC       XR        118034.0      1
M-LC       XR        106986.0      1
L-SC       OX40N      90555.0      1
M-SC       X40        87940.0      1
H-LC       X200       74819.0      1
L-MC       XR         58634.0      1
H-SC       MH         56027.0      1
H-MC       OX40N      48696.0      1
L-LC       XY25       44517.0      1
H-MC       AR         42926.0      1
H-LC       XY24       40954.0      1
H-MC       MH         32168.0      1
H-LC       X5K        10282.0      1
M-LC       XY25        9470.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
